Копируем в данный файл код наивной байесовской классификации

In [1]:
import pandas as pd
from collections import defaultdict
from math import log
from sklearn.model_selection import train_test_split

def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        for feat in feats:
            freq[label, feat] += 1          # count features frequencies

    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)
    return classes, freq                    # return P(C) and P(O|C)

def classify(classifier, feats):
    classes, prob = classifier
    return min(classes.keys(),              # calculate argmin(-log(C|O)) -> argmax(P(C|O))
        key = lambda cl: -log(classes[cl]) + \
            sum(-log(prob.get((cl,feat), 1/1000000000)) for feat in feats))

def get_features(sample): return (sample[-1]) # get last letter

Открываем файл "baby-names.csv" и перемешиваем данные, так как в dataframe сначала идут мальчики, а затем девочки

In [2]:
data = pd.read_csv("baby-names.csv")

Разделяем данные в выборке на обучающие и тестовые в соотношении 70%/30%

In [3]:
x_train, x_test, y_train, y_test = train_test_split(data['name'], data['sex'], train_size=0.7)

Обучаем наивную байесовскую классификацию из файла Sem2.ipynb на тренировочном наборе данных. Затем с помощью
метода classify() размечаем имена по полу в тестировочном наборе данных

In [4]:
features = [(get_features(x_train[index]), y_train[index]) for index, row in x_train.items()]
classifier = train(features)
pred = [classify(classifier, get_features(row)) for index, row in x_test.items()]

Посчитаю процент "правильных ответов" классификатора

In [5]:
from sklearn.metrics import accuracy_score

print("Процент правильных ответов - ", round(accuracy_score(y_test, pred)*100, 2), "%")

Процент правильных ответов -  77.83 %


Также еще можно построить матрицу ошибок (одна из метрик), чтобы оценить, насколько хорошо 
справился с задачей данный классификато и где именно он ошибся

In [6]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, pred)

print(f'Правильно определенных мальчиков - {matrix[0][0]}. В процентах от мальчиков - {round(matrix[0][0]/(matrix[0][0] + matrix[0][1])*100, 3)}%')
print(f'Неправильно определенных мальчиков - {matrix[0][1]}. В процентах от мальчиков - {round(matrix[0][1]/(matrix[0][0] + matrix[0][1])*100, 3)}%')
print(f'Правильно определенных девочек - {matrix[1][1]}. В процентах от девочек - {round(matrix[1][1]/(matrix[1][0] + matrix[1][1])*100, 3)}%')
print(f'Неправильно определенных девочек - {matrix[1][0]}. В процентах от девочек - {round(matrix[1][0]/(matrix[1][0] + matrix[1][1])*100, 3)}%')

Правильно определенных мальчиков - 32478. В процентах от мальчиков - 83.907%
Неправильно определенных мальчиков - 6229. В процентах от мальчиков - 16.093%
Правильно определенных девочек - 27763. В процентах от девочек - 71.752%
Неправильно определенных девочек - 10930. В процентах от девочек - 28.248%


Как можно видеть, классификатор практически в два раза чаще ошибался в определение девочек и ошибочно принимал их за мальчиков.

Модифицирую функцию get_features() таким образом, чтобы в качестве целевого признака бралась другая структура. Попробовав одну первую букву, две первые буквы, первую и последнюю букву вместе, имя целиком, последние две буквы, могу сделать вывод, что вариант с первой и последней буквой наилучший, но все равно практически не отличается от варианта просто с последней буквой

In [7]:
def get_features_new(sample): return (sample[0], sample[-1])

features = [(get_features_new(x_train[index]), y_train[index]) for index, row in x_train.items()]
classifier = train(features)
pred = [classify(classifier, get_features_new(row)) for index, row in x_test.items()]
print("Процент правильных ответов - ", round(accuracy_score(y_test, pred)*100, 2), "%")

Процент правильных ответов -  78.29 %


Модифицирую метод classify() так, чтобы вместо логарифмов бралась функция -1/(исходные значения вероятностей), а вместо argmin(...) считался соответственно функционал argmax (..) . Перебрав несколько функций, выяснил, что данная является оптимальной и сравнима с логарифмической.

In [8]:
def classify_new(classifier, feats):
    classes, prob = classifier
    return max(classes.keys(),              # calculate argmin(-log(C|O)) -> argmax(P(C|O))
        key = lambda cl: -1/(classes[cl]) + \
            sum(-1/prob.get((cl,feat), 1/1000000000) for feat in feats))

features = [(get_features(x_train[index]), y_train[index]) for index, row in x_train.items()]
classifier = train(features)
pred = [classify_new(classifier, get_features(row)) for index, row in x_test.items()]
print("Процент правильных ответов - ", round(accuracy_score(y_test, pred)*100, 2), "%")

Процент правильных ответов -  77.83 %


Доля правильных ответов алгоритма после модификации целевого признака и изменения метода classify() практически не изменилась, но если использовать другие варианты, то результат значительно ухудшается (до 50-60%). Из чего можно сделать выводы, чтоо выбор целевых признаков и классифицирующей 
функци очень сильно влияет на результат алгоритм

Запущу гауссовский классификатор методом из sklearn.naive_bayes и посмотрю процент "правильных ответов" на тестовой выборке

In [9]:
from sklearn.naive_bayes import GaussianNB

x_train = x_train.map(lambda x: ord(get_features(x))) # Отбираем последнюю букву в обучающую выборке
x_train = x_train.to_numpy().reshape(-1, 1)   # Преобразуем обучающую выборку к массиву numpy
y_train = y_train.to_numpy()

x_test = x_test.map(lambda x: ord(get_features(x)))
x_test = x_test.to_numpy().reshape(-1, 1)

gnb = GaussianNB()
gnb.fit(x_train, y_train)

y_predict = gnb.predict(x_test)
print("Процент правильных ответов - ", round(accuracy_score(y_test, y_predict)*100, 2), "%")

Процент правильных ответов -  73.26 %


Как можно видеть, процент "правильных" ответов стал меньше, чем в случае наивного байесовского классификатора.
Теперь запущу мультиномиальный классификатор методом из sklearn.naive_bayes и посмотрю процент "правильных ответов" на тестовой выборке

In [10]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_predict = clf.predict(x_test)
print("Процент правильных ответов - ", round(accuracy_score(y_test, y_predict)*100, 2), "%")

Процент правильных ответов -  49.99 %


Процент правильных ответов стал еще ниже, то есть мультиномиальный классификатор хуже остальных справляется с данной задачей (правильнее даже отметить, что он выдает ответы наугад, так как процент правильных ответов 50% ).
В итоге, можно сделать вывод, что самым эффективным классификатором оказался наивный байесовский классификатор. Самым неэффективный классификатор -  мультиномиальный и, соответственно, средний из них гауссовский.